# Natural Gas Storage Contract's Pricer #
The goal of this tool is to give a price range for which a full contract can be sold to a client, and still be profitable for the company.

#### JP Morgan Instructions : ####
The input parameters that should be taken into account for pricing are:
1. Injection dates. 
2. Withdrawal dates.
3. The prices at which the commodity can be purchased/sold on those dates.
4. The rate at which the gas can be injected/withdrawn.
5. The maximum volume that can be stored.
6. Storage costs.

### Setting the Libraries and preparing the data ###

In [40]:
# Importing Libraries
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
from scipy.stats import norm
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, date
import math

In [41]:
# Importing and treating the data
gas_df = pd.read_csv('natural_gas_market_data.csv')
gas_df['Dates'] = pd.to_datetime(gas_df['Dates'])
gas_df.set_index('Dates', inplace=True)
gas_df.head()

/var/folders/0v/nd2019hd3lxb6r07g9ywbv500000gn/T/ipykernel_94243/4155221353.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  gas_df['Dates'] = pd.to_datetime(gas_df['Dates'])


,Prices
Dates,
2020-10-31,10.1
2020-11-30,10.3
2020-12-31,11.0
2021-01-31,10.9
2021-02-28,10.9


In [42]:
# Redefining the forecasting function from task 1
def forecast_gas_prices(gas_df):
    
    from statsmodels.tsa.statespace.sarimax import SARIMAX
    
    # Splitting the data into training and testing sets
    train_data = gas_df[:len(gas_df)-12]
    test_data = gas_df[len(gas_df)-12:]
    
    # Compiling the model and saving it
    arima_model = SARIMAX(train_data['Prices'], order=(2,1,2), seasonal_order=(1,1,1,12), verbose=0)
    arima_result = arima_model.fit()
    # arima_result.summary()
    
    # Predicting the next 12 months
    arima_pred_12m = arima_result.predict(start=len(gas_df), end=len(gas_df)+12, type='levels', verbose=0).rename('ARIMA Predictions')
    
    # Convert `arima_pred_12m` (Series) to DataFrame
    arima_pred_12m = arima_pred_12m.to_frame(name='forecast')
    
    # Reindex `arima_pred_12m` to align with `gas_df`
    forecast_df = pd.DataFrame(index=arima_pred_12m.index)
    forecast_df['forecast'] = arima_pred_12m['forecast']
    forecast_df['Prices'] = float('NaN')  # Fill Prices with NaN

    return forecast_df

forecast_df = forecast_gas_prices(gas_df)

/Users/thibaultmarty/miniconda3/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency ME will be used.
  self._init_dates(dates, freq)
/Users/thibaultmarty/miniconda3/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency ME will be used.
  self._init_dates(dates, freq)
/Users/thibaultmarty/miniconda3/lib/python3.12/site-packages/statsmodels/tsa/statespace/representation.py:374: FutureWarning: Unknown keyword arguments: dict_keys(['verbose']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.
  warnings.warn(msg, FutureWarning)
/Users/thibaultmarty/miniconda3/lib/python3.12/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('T

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            7     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.63074D-01    |proj g|=  1.65386D-01

At iterate    5    f=  7.81620D-02    |proj g|=  1.75871D-01

At iterate   10    f=  4.61098D-02    |proj g|=  1.34652D-01

At iterate   15    f=  4.24090D-02    |proj g|=  5.41430D-02

At iterate   20    f=  1.52570D-02    |proj g|=  2.31647D-01

At iterate   25    f=  2.03654D-03    |proj g|=  6.46259D-02

At iterate   30    f= -3.65749D-03    |proj g|=  2.00373D-02

At iterate   35    f= -4.49468D-03    |proj g|=  6.66961D-02

At iterate   40    f= -4.59249D-03    |proj g|=  7.43199D-03

At iterate   45    f= -4.59397D-03    |proj g|=  3.21794D-03

At iterate   50    f= -4.59412D-03    |proj g|=  6.31825D-05

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cau

/Users/thibaultmarty/miniconda3/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/thibaultmarty/miniconda3/lib/python3.12/site-packages/statsmodels/tsa/statespace/representation.py:374: FutureWarning: Unknown keyword arguments: dict_keys(['type', 'verbose']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.
  warnings.warn(msg, FutureWarning)


In [43]:
# Obtaining the full dataset
gas_df_combined = pd.concat([gas_df, forecast_df])
print(gas_df_combined)

            Prices   forecast
2020-10-31    10.1        NaN
2020-11-30    10.3        NaN
2020-12-31    11.0        NaN
2021-01-31    10.9        NaN
2021-02-28    10.9        NaN
...            ...        ...
2025-06-30     NaN  11.950996
2025-07-31     NaN  12.352083
2025-08-31     NaN  12.121968
2025-09-30     NaN  12.496624
2025-10-31     NaN  12.273729

[61 rows x 2 columns]


In [44]:
# Function to find the quote for a given date
def get_gas_quote(date):
    '''
    Function that returns the quote for a given date, using known quotes
    and estimated quotes from the SARIMAX model.
    Inputs:
        date: date for which we want to find the quote
    Outputs:
        quote: quote for the given date
    '''
    # Convert date into a datetime object
    input_date = pd.to_datetime(date)

    # Check if the date is within the DataFrame index
    if input_date in gas_df_combined.index:
        # Check if 'Prices' is NaN for the input date
        if pd.isna(gas_df_combined.loc[input_date, 'Prices']):
            # Return the forecasted value
            return gas_df_combined.loc[input_date, 'forecast']
        else:
            # Return the actual price
            return gas_df_combined.loc[input_date, 'Prices']
    else:
        print('Date not found in the quotes.')
        return None


In [45]:
quote = get_gas_quote('2025-07-31')
quote

12.35208326382871

In [61]:
# Function to model and compute the contract price
def compute_contract_price(in_dates, in_prices, out_dates, out_prices, 
                           rate, storage_cost_rate, total_vol, 
                           injection_withdrawal_cost_rate):
    volume = 0
    buy_cost = 0
    cash_in = 0
    last_date = min(min(in_dates), min(out_dates))
    
    # Ensure dates are in sequence
    all_dates = sorted(set(in_dates + out_dates))
    
    for i in range(len(all_dates)):
        # processing code for each date
        start_date = all_dates[i]

        if start_date in in_dates:
            # Inject on these dates and sum up cash flows
            if volume <= total_vol - rate:
                volume += rate

                # Cost to purchase gas
                buy_cost += rate * in_prices[in_dates.index(start_date)]
                # Injection cost
                injection_cost = rate * injection_withdrawal_cost_rate/1000000
                buy_cost += injection_cost
                print(f'Injected gas on {start_date.strftime('%Y-%m-%d')} at a price of {in_prices[in_dates.index(start_date)]}$')

            else:
                # We do not want to inject when rate is greater than total volume minus volume
                print(f'Injection is not possible on date {start_date.strftime('%Y-%m-%d')} as there is insufficient space in the storage facility')
        
        elif start_date in out_dates:
            #Withdraw on these dates and sum cash flows
            if volume >= rate:
                volume -= rate
                cash_in += rate * out_prices[out_dates.index(start_date)]
                # Withdrawal cost
                withdrawal_cost = rate * injection_withdrawal_cost_rate/1000000
                cash_in -= withdrawal_cost
                print(f'Extracted gas on {start_date.strftime('%Y-%m-%d')} at a price of {out_prices[out_dates.index(start_date)]}$')
            else:
                # we cannot withdraw more gas than is actually stored
                print(f'Extraction is not possible on date {start_date.strftime('%Y-%m-%d')} as there is insufficient volume of gas stored')
    store_cost = math.ceil((max(out_dates) - min(in_dates)).days // 30) * storage_cost_rate
    return cash_in - buy_cost - store_cost

In [63]:
# Example usage of price_contract()
in_dates = [date(2021, 7, 31), date(2021, 11, 30), date(2022, 7, 31), date(2023, 6, 30)] #injection dates
in_prices = [round(get_gas_quote(in_date),2) for in_date in in_dates]#prices on the injection days
out_dates = [date(2022, 3, 31), date(2023, 2, 28), date(2024, 1, 31), date(2025, 2, 28)] # extraction dates
out_prices = [round(get_gas_quote(out_date),2) for out_date in out_dates] # prices on the extraction days

# Commodity storage data
rate = 100000  # MMBtu/month
storage_cost_rate = 10000  # storage cost rate $/MMBtu/month
injection_withdrawal_cost_rate = 10000  # $/MMBtu
max_storage_volume = 50000000 # maximum storage capacity of the storage facility

result = compute_contract_price(in_dates, in_prices, out_dates, out_prices, rate, storage_cost_rate, max_storage_volume, injection_withdrawal_cost_rate)
print()
print(f"The value of the contract is: ${result}")

Injected gas on 2021-07-31 at a price of 10.1$
Injected gas on 2021-11-30 at a price of 11.2$
Extracted gas on 2022-03-31 at a price of 11.5$
Injected gas on 2022-07-31 at a price of 10.5$
Extracted gas on 2023-02-28 at a price of 11.7$
Injected gas on 2023-06-30 at a price of 10.9$
Extracted gas on 2024-01-31 at a price of 12.6$
Extracted gas on 2025-02-28 at a price of 12.91$

The value of the contract is: $163000.0
